In [ ]:
from tensorflow.keras.utils import save_img
from matplotlib import pyplot as plt
from os import mkdir, path
import tensorflow as tf
import numpy as np
import pathlib
import glob
import cv2


dataset = 'hk_dataset' # 'hk_dataset' or 'miche_dataset'
hk_validation = True
mirror = False 
model_selection = True 

DATASET_PATH = 'task_1/dataset/processed_mirror_hk_dataset_320x256_ac_4/'
MODELS_PATH = 'task_1/transcoding/models/'
GENERATED_IMAGES_PATH = 'task_1/transcoding/results/hk_dataset/'

# parameters for miche evaluation
right_eye_miche_subjects = [] # fill the array with subjects who has right eye acquisitions


models_list = [ 'best_pix2pix_bs4_vggloss3_L_100', 
                'best_unet_bs4_l1_loss' ]


IMG_WIDTH = 320
IMG_HEIGHT = 256
LENGTH_IMAGE_NAME = 12 

# real image size
hk_real_img_width = 640
hk_real_img_height = 480

In [ ]:
if dataset == 'hk_dataset' :
    f = 'test/*.png'
    LENGTH_IMAGE_NAME = 12 

elif dataset == 'miche_dataset' :
    f = '/*.jpg'
    LENGTH_IMAGE_NAME = 24 
    

if hk_validation : 
    testset_files = glob.glob(DATASET_PATH + 'test/*.png')
    testset_files.sort()

    new_test_files = []
    len_file = len(testset_files[1])

    i=0
    for file in testset_files:
        sub = testset_files[i][len_file-11:len_file-8]
        if model_selection :
            if int(sub) <= 180 :
                new_test_files.append(file)
        else : 
            if int(sub) > 180 :
                new_test_files.append(file)
        i = i+1 
        
    testset_files = new_test_files
    testset_size = len(testset_files)

else :
    
    testset_files = glob.glob(DATASET_PATH + f)
    testset_files.sort()
    testset_size = len(testset_files)


PATHLIB_DATASET_PATH  = pathlib.Path(DATASET_PATH)
LENGTH_IMAGE_PATH = len(testset_files[0])

In [ ]:
def load(image_file):
    # Read and decode an image file to a uint8 tensor
    image = tf.io.read_file(image_file)
    #image = tfio.experimental.image.decode_tiff(image)
    #r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    #image = tf.stack([r, g, b], axis=-1)
    

    if dataset == 'hk_dataset' : 
        image = tf.io.decode_png(image)

        # Split each image tensor into two tensors:
        # - one with a real building facade image
        # - one with an architecture label image 
        w = tf.shape(image)[1]
        w = w // 2

        input_image = image[:, :w, :]
        real_image = image[:, w:, :]

        # Convert both images to float32 tensors
        input_image = tf.cast(input_image, tf.float32)
        real_image = tf.cast(real_image, tf.float32)

    elif dataset == 'new_miche_dataset' :
        image = tf.io.decode_png(image)

        input_image = image
        real_image = []
        
        # Convert both images to float32 tensors
        input_image = tf.cast(input_image, tf.float32)          

    return input_image, real_image

In [ ]:
def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    if dataset == 'hk_dataset' :
        real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    return input_image, real_image

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    
    if dataset == 'hk_dataset' :
        real_image = (real_image / 127.5) - 1
         
    return input_image, real_image

In [ ]:
def generate_images(model, inp, tar):
    prediction = model(inp, training=False)
    plt.figure(figsize=(15, 15))

    display_list = [tar[0], prediction[0]]
    title = ['Ground Truth', 'Predicted Image']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        
        # Getting the pixel values in the [0, 1] range to plot.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
def print_images(gen_output, inp):

    plt.figure(figsize=(15, 15))
    display_list = [inp[0], gen_output[0]]
    title = ['Ground Truth', 'Predicted Image']
    
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])

        # Getting the pixel values in the [0, 1] range to plot.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()


In [ ]:
def save_image(image, images_path, i):
    file_name = testset_files[i][LENGTH_IMAGE_PATH-LENGTH_IMAGE_NAME:LENGTH_IMAGE_PATH]  
    file_path = images_path + '/' + file_name 

    img = image[0].numpy()
    
    if dataset == 'hk_dataset' and not mirror and file_name[5] == 'R':
        img = img[:, ::-1, :]

    img = cv2.normalize(img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    img = img.astype(np.uint8)

    if dataset == 'miche_dataset' :
        real_img_path = DATASET_PATH + file_name
        real_img = cv2.imread(real_img_path)
        real_img_height = real_img.shape[0]
        real_img_width = real_img.shape[1]

        subj = int(file_name[0:3])
        if (subj in right_eye_miche_subjects) and not mirror : 
            img = img[:, ::-1, :]

        img = cv2.resize(img, (real_img_width,real_img_height))
    
    elif dataset == 'hk_dataset' :
        img = cv2.resize(img, (hk_real_img_width ,hk_real_img_height))
   
    cv2.imwrite(file_path, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


In [ ]:
# printing img testset 

inp, re = load(testset_files[1])
inp, re = resize(inp,re, IMG_HEIGHT, IMG_WIDTH)
inp, re = normalize(inp,re)
plt.figure(figsize=(6, 6))

display_list = [inp]
title = ['Ground Truth']

for i in range(1):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i]* 0.5 + 0.5)
    plt.axis('off')

plt.show()

In [ ]:
def load_image_test(image_file):
    
    input_image, real_image = load(image_file)
    input_image, real_image = normalize(input_image, real_image)

    if not dataset == 'hk_dataset' :
        input_image, real_image = resize(input_image, real_image, IMG_HEIGHT, IMG_WIDTH)

    return input_image, real_image

In [ ]:
if dataset == 'hk_dataset' :
    f = 'test/*.png'
    
elif dataset == 'miche_dataset':
    f = '*.jpg'

if hk_validation :
    test_dataset = tf.data.Dataset.from_tensor_slices(testset_files)
    test_dataset = test_dataset.map(load_image_test)
    test_dataset = test_dataset.batch(1)
else :
    test_dataset = tf.data.Dataset.list_files(str(PATHLIB_DATASET_PATH / f), shuffle=False)
    test_dataset = test_dataset.map(load_image_test)
    test_dataset = test_dataset.batch(1)

In [ ]:
def generator_loss(target, gen_output):
    # PSNR 
    psnr_loss = tf.image.psnr(target, gen_output, max_val=2.0) # images shuld have been normalized in range [-1,1]. Thus, the difference between the min and max should be 2.
    
    # SSIM 
    ssim_loss = tf.image.ssim(target, gen_output, max_val=2.0)

    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    # Mean squared error
    l2_loss = tf.reduce_mean(tf.abs(target - gen_output)**2)
    
    return psnr_loss.numpy().item(0), ssim_loss.numpy().item(0), l1_loss.numpy(), l2_loss.numpy()

In [ ]:
for model_name in models_list :
    
    generator = tf.keras.models.load_model(MODELS_PATH + model_name + '.h5', compile=False)
    model_images_path = GENERATED_IMAGES_PATH + model_name

    mkdir(model_images_path)

    sum_l1_losses = [] 
    sum_l2_losses = []
    sum_psnr_losses = []
    sum_ssim_losses = []

    n_image = 0
    for inp, tar in test_dataset :
        gen_output = generator(inp, training=False)

        #print_images(gen_output, inp)

        if dataset == 'hk_dataset' : 
            psnr_loss, ssim_loss, l1_loss, l2_loss = generator_loss(tar, gen_output)

            sum_psnr_losses.append(psnr_loss)
            sum_ssim_losses.append(ssim_loss)
            sum_l1_losses.append(l1_loss)
            sum_l2_losses.append(l2_loss)
        
        save_image(gen_output, model_images_path, n_image)
        n_image = n_image + 1

    if dataset == 'hk_dataset' : 
        print('\t' + model_name)
        print("\t  PSNR loss")
        print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_psnr_losses)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_psnr_losses)))
        print("\t  SSIM loss")
        print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_ssim_losses)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_ssim_losses)))
        print("\t  L1 loss - L2 loss")
        print("\t\t  l1_loss  : " +  "{:.2f}".format(np.mean(sum_l1_losses)) +  "\tl2_loss : " +  "{:.2f}".format(np.var(sum_l1_losses)))
        print("\t\t  l2_loss  : " +  "{:.2f}".format(np.mean(sum_l2_losses)) +  "\tl2_loss : " +  "{:.2f}".format(np.var(sum_l2_losses)))

        print("\n\n")
